# VARIMA model

## Imports

In [ ]:
import random

import torch
torch.cuda.is_available()

In [ ]:
import optuna
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

from darts import TimeSeries
from darts.models import VARIMA
from darts.metrics import rmse
from darts.dataprocessing.transformers import Scaler, Diff
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.statistics import plot_acf, plot_pacf, stationarity_test_kpss, stationarity_test_adf

from time_series_model.data.weather.weather_dataloader import MeteostatDataLoader
from time_series_model.data.data_loading import SMARDDataLoader
from time_series_model.evaluation_method import get_covariate_args, cross_validation_without_refit

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("time_series_model").setLevel(logging.INFO)
logging.getLogger('lightning').setLevel(0)
logging.getLogger('pytorch_lightning').setLevel(0)
logging.getLogger('darts').setLevel(0)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Load SMARD data

In [ ]:
smard_dataloader = SMARDDataLoader(
    file_paths=[
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', '2015_2016.csv'),
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', '2017_2018.csv'),
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', '2019_2020.csv'),
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', '2021_2022.csv'),
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', '2022_2023.csv')
    ]
)
smard_dataloader.load_data()
smard_dataloader.preprocess_data()
smard_dataloader.validate_data()

smard_data_df: DataFrame = smard_dataloader.data
smard_data_df['timestamp'] = pd.to_datetime(smard_data_df['timestamp'])
smard_data_df = smard_data_df.set_index('timestamp')
smard_data = TimeSeries.from_dataframe(
    smard_data_df, 
    value_cols=list(smard_data_df.columns), 
    fill_missing_dates=True, 
    fillna_value=0, 
    freq='1H'
)

In [ ]:
smard_data_df.describe()

In [ ]:
smard_data_df.hist(figsize=(10,20), bins=30)

### probe order of differencing

In [ ]:
for resource in smard_data.columns:
    plot_acf(smard_data[resource].diff(), max_lag=24*7)
    plot_acf(smard_data[resource], max_lag=24*7)
    plt.show()

## load weather data (from solar & wind stations)

In [ ]:
meteostat_solar_loader = MeteostatDataLoader(
    file_paths=[
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', 'weather_data_solar_stations.csv')
    ],
    solar=True
)
meteostat_solar_loader.load_data()
meteostat_solar_loader.preprocess_data()
solar_data = meteostat_solar_loader.data


meteostat_wind_loader = MeteostatDataLoader(
    file_paths=[
        os.path.join(os.getcwd(), os.pardir, 'data', 'raw', 'weather_data_wind_stations.csv')
    ],
    wind=True
)
meteostat_wind_loader.load_data()
meteostat_wind_loader.preprocess_data()
wind_data = meteostat_wind_loader.data


print("Missing values for wind")
for col in wind_data.columns:
    print(f"  Column {col} has {wind_data[col].isna().mean() * 100:0.2f}% missing values")
print("Missing values for solar")
for col in solar_data.columns:
    print(f"  Column {col} has {solar_data[col].isna().mean() * 100:0.2f}% missing values")

solar_data['time'] = pd.to_datetime(solar_data['time'])
wind_data['time'] = pd.to_datetime(wind_data['time'])

solar_data = solar_data.set_index('time')
wind_data = wind_data.set_index('time')

print(f"Wind data index: {wind_data.index.min()} - {wind_data.index.max()}")
print(f"Solar data index: {solar_data.index.min()} - {solar_data.index.max()}")

solar_data = TimeSeries.from_dataframe(
    solar_data, 
    value_cols=list(solar_data.columns), 
    fill_missing_dates=True, 
    fillna_value=0, 
    freq='1H'
)
wind_data = TimeSeries.from_dataframe(
    wind_data, 
    value_cols=list(wind_data.columns), 
    fill_missing_dates=True, 
    fillna_value=0, 
    freq='1H'
)

wind_data = wind_data.astype(np.float32)
solar_data = solar_data.astype(np.float32)

weather_data = solar_data.stack(wind_data)

In [ ]:
weather_data[-24*7:].plot()

In [ ]:
smard_data[-24*7:].plot()

## create time covariates

In [ ]:
weekday = datetime_attribute_timeseries(weather_data, attribute="weekday", dtype=np.float32)
month = datetime_attribute_timeseries(weather_data, attribute="month", dtype=np.float32)
hour = datetime_attribute_timeseries(weather_data, attribute="hour", dtype=np.float32)
covariates_time = weekday.stack(hour).stack(month)

scaler_covariates = Scaler()
covariates_time = scaler_covariates.fit_transform(
    covariates_time
)

In [ ]:
plt.figure(figsize=(12,2))
covariates_time[-5*7*24:].plot()
plt.title("Covariates Time")

## differencing of target series

In [ ]:
smard_differencing_transformer = Diff()
differenced_smard_data = smard_differencing_transformer.fit_transform(smard_data)

In [ ]:
differenced_smard_data[-24*30:].plot()

In [ ]:
smard_data[-24*30:].plot()

## train, val & test split of SMARD data

In [ ]:
test_split = 0.15
val_split=0.15
train_end_index = (1 - val_split - test_split) * len(differenced_smard_data)
val_end_index = (1 - test_split) * len(differenced_smard_data)
train_end_index, val_end_index = int(train_end_index), int(val_end_index)

print(train_end_index, val_end_index)

train, val, test = differenced_smard_data[:train_end_index], differenced_smard_data[train_end_index:val_end_index], differenced_smard_data[val_end_index:]

# Cut train to start of weather data
train = train[weather_data.start_time():]

print("Train size: ", len(train))
print("Val size: ", len(val))
print("Test size: ", len(test))

## train model on last week of training data

In [ ]:
def fit_model(model: VARIMA, train: TimeSeries, covariates: TimeSeries):
    covariate_args = get_covariate_args(
        model=model,
        covariates=covariates,
    )[0]
    model.fit(series=train, future_covariates=covariate_args['future_covariates'])
    model.save()

In [ ]:
varima_model = VARIMA()
last_n_samples = -24*7
fit_model(varima_model, train[last_n_samples:], covariates_time.stack(weather_data)[train[last_n_samples:].start_time():])

In [ ]:
def predict_model(model: VARIMA, n: int, covariates: TimeSeries):
    covariate_args = get_covariate_args(
        model=model,
        covariates=covariates,
    )[0]
    return model.predict(n=n, future_covariates=covariate_args['future_covariates'])

## predict the next 24 hours

In [ ]:
differenced_predictions = predict_model(varima_model, 24, covariates_time.stack(weather_data)[train[last_n_samples:].start_time():])
differenced_predictions.start_time(), differenced_predictions.end_time()

In [ ]:
differenced_predictions.components

In [ ]:
val.start_time(), val.end_time()

### invert differenced predictions & validation set

In [ ]:
predictions = smard_differencing_transformer.inverse_transform(train.append(differenced_predictions))[differenced_predictions.start_time():]

In [ ]:
predictions

Note: since `inverse_transform()` renames components of `differenced_predictions` and `val` we have to manually reset them

In [ ]:
def replace_components(series: TimeSeries, columns: pd.Index) -> TimeSeries:
    series_df = series.pd_dataframe()
    series_df.columns = columns
    return TimeSeries.from_dataframe(series_df)

In [ ]:
predictions = replace_components(predictions, differenced_predictions.columns)
predictions

In [ ]:
val_inverted_trimmed = smard_differencing_transformer.inverse_transform(train.append(val))[predictions.start_time():predictions.end_time()]
val_inverted_trimmed.start_time(), val_inverted_trimmed.end_time()

In [ ]:
val_inverted_trimmed = replace_components(val_inverted_trimmed, differenced_predictions.columns)
val_inverted_trimmed

### plot predictions & actual values and calculate RMSE

In [ ]:
for prediction_column, val_column in zip(predictions.columns, val_inverted_trimmed.columns):
    assert prediction_column == val_column
for prediction_column, val_column in zip(predictions.columns, val_inverted_trimmed.columns):
    plt.figure()
    predictions[prediction_column].plot(label='predictions')
    val_inverted_trimmed[val_column].plot(label='actual')
    plt.title(prediction_column)
    plt.show()
print(f'RMSE: {rmse(val_inverted_trimmed, predictions)}')

## optimize VARIMA model via optuna

In [ ]:
# TODO: adjust this parameter for training duration
# it simply takes the last n samples of the training data as actual training data since VARIMA training lasts quite long on CPU
last_n_samples = 3*24

# optimizing function
def objective(trial):

    p = trial.suggest_int('p', 0, 2)
    d = trial.suggest_int('d', 0, 1)
    q = trial.suggest_int('q', 0, 2)
    if p == 0 and q == 0:
        p = random.randint(0, 1)
        q = 1 - p
    trend = trial.suggest_categorical('trend', ['n', 'c', 't', 'ct', None])

    print(f"Trialing with {trial.params}")

    optimizing_model = VARIMA(p=p, d=d, q=q, trend=trend)

    fit_model(optimizing_model, train[-last_n_samples:], covariates_time.stack(weather_data)[train[-last_n_samples:].start_time():])

    eval_result = cross_validation_without_refit(
        model=optimizing_model,
        series=train.concatenate(val),
        start=val.start_time(),
        metrics=[rmse],
        data_scaler=smard_differencing_transformer,
        covariates=covariates_time.stack(weather_data),
        max_n_split=5,
        forecast_horizon=24,
        plotting=False
    )

    eval_rmse = eval_result['rmse']
    print(f"Eval RMSE: {eval_rmse}")

    return eval_rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

In [ ]:
study.best_params

In [ ]:
best_params = study.best_params
print(f"Best params: {best_params}")

final_model = VARIMA(**best_params)

# TODO: adjust this parameter like above
last_n_samples = 3*24
fit_model(model=final_model, train=train.concatenate(val)[-last_n_samples:], covariates=covariates_time.stack(weather_data)[train.concatenate(val)[-last_n_samples:].start_time():])

# Evaluate
eval_result = cross_validation_without_refit(
    model=final_model,
    series=train.concatenate(val.concatenate(test)),
    start=test.start_time(),
    metrics=[rmse],
    data_scaler=smard_differencing_transformer,
    covariates=covariates_time.stack(weather_data),
    max_n_split=5,
    forecast_horizon=24,
    plotting=False
) 

eval_rmse = eval_result['rmse']
print(f"Eval RMSE: {eval_rmse}")